In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
link = 'https://fapergs.rs.gov.br/encerrados'

links_estados = []

parser_links = parser(link)

parser_links.xpath('//text()')

driver = webdriver.Firefox()
driver.get(link)

links_bruto = driver.find_elements_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//h2//a')

for link in links_bruto:
    links_estados.append(link.get_attribute('href'))

driver.quit()

links_estados.append('https://fapergs.rs.gov.br/em-julgamento')
links_estados.append('https://fapergs.rs.gov.br/abertos')

In [ ]:

#######################
#parser_lista_proyectos = parser()
fapergs = pd.DataFrame()

for link in links_estados:

    links_proyectos = []
    titulos_proyectos = []

    fapergs_estado = pd.DataFrame()

    driver = webdriver.Firefox()
    driver.get(link)

    bases = driver.find_elements_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//h2//a')

    for base in bases:
        links_proyectos.append(base.get_attribute('href'))
        titulos_proyectos.append(base.text)

    estado = driver.find_element_by_xpath('//article[@class="conteudo-lista__item clearfix"]//header//ul//li//a').text

    ano = driver.find_element_by_xpath('//article//header[@class="artigo__cabecalho"]//h1').text
    if ano == 'Em Julgamento' or ano == 'Abertos':
        ano = '2021'
    
    driver.quit()

    fapergs_estado['Título'] = titulos_proyectos
    fapergs_estado['Link'] = links_proyectos
    fapergs_estado['Año'] = ano
    fapergs_estado['Estado'] = estado

    fapergs = pd.concat([fapergs, fapergs_estado])

fapergs = fapergs.reset_index(drop=True) 

In [ ]:
pdfs_proyectos = []
descripcion_proyectos = []

for link in fapergs['Link']:
    driver = webdriver.Firefox()
    driver.get(link)
    ## PDFs
    pdfs = driver.find_elements_by_xpath('//div[@class="row"]//a')

    pdfs_proy = ''
    for p in pdfs:
        
        p = str(p.get_attribute('href'))
        
        if p.endswith('.pdf'):
            pdfs_proy = pdfs_proy + p + ', '

    pdfs_proyectos.append(pdfs_proy.strip(', '))

    #Descripción

    descripcion_bruta = driver.find_elements_by_xpath('//div[@class="row"]//p')
    descripcion = ''
    for d in descripcion_bruta:
        
        d = d.text
        descripcion = descripcion + d + ' '

    descripcion_proyectos.append(descripcion)

    driver.quit()
#print(pdfs_proyectos)
#print(descripcion_proyectos)
fapergs['Pdfs'] = pdfs_proyectos
fapergs['Descripción'] = descripcion_proyectos

In [ ]:
fapergs.to_excel('brasil_fapergs.xlsx')